# cvc5 SMT求解器：演示

---

## 目录

**第一部分：基础（1-6）**
- 1. cvc5简介与安装
- 2. Hello cvc5
- 3. SAT求解专题
- 4. 整数规划
- 5. 量词逻辑
- 6. 数组理论

**第二部分：应用（7-9）**
- 7. 数独求解器
- 8. N皇后问题
- 9. 有界模型检查(BMC)
  - 9.1 实战：硬件计数器验证

**第三部分：深度剖析（10-16）**
- 10. DPLL(T)算法内部机制
- 11. UNSAT Core冲突分析
- 12. 增量求解与状态管理
- 13. 位向量硬件验证
- 14. 量词实例化优化
- 15. 程序正确性验证
- 16. 递归数据结构验证

**第四部分：创新应用（17）** 
- 17. LLM + cvc5 自然语言约束求解器
  - 示例1: 整数约束问题
  - 示例2: 逻辑推理问题
  - 示例3: 数学问题
  - 技术实现细节
---

## 1. cvc5简介与安装

**cvc5**是一个SMT (Satisfiability Modulo Theories) 求解器。

SMT = SAT + 理论求解器

```
SAT:  只处理布尔变量 (True/False)
SMT:  布尔变量 + 整数 + 实数 + 数组 + 位向量 + ...
```

**安装：**
```bash
pip install cvc5
```

**应用场景：**
- 程序验证（证明代码正确性）
- 硬件验证（验证芯片设计）
- 约束求解（配置管理、调度问题）

In [33]:
import cvc5
print("cvc5版本:", cvc5.__version__)


cvc5版本: 1.3.1


---

## 2. Hello cvc5：第一个约束求解

**问题：** 找整数x, y使得 x + y = 5, x > 0, y > 0

**步骤：**
1. 创建变量
2. 添加约束
3. 调用求解器
4. 获取解

In [2]:
print("="*60)
print("示例：Hello cvc5")
print("="*60)

# 1. 创建整数变量
x, y = Ints('x y')

# 2. 创建求解器并添加约束
solver = Solver()
solver.add(x + y == 5)
solver.add(x > 0)
solver.add(y > 0)

# 3. 求解
result = solver.check()
print(f"结果: {result}")

# 4. 获取模型
if result == sat:
    model = solver.model()
    x_val = model[x].as_long()
    y_val = model[y].as_long()
    print(f"解: x = {x_val}, y = {y_val}")
    print(f"验证: {x_val} + {y_val} = {x_val + y_val}")

示例：Hello cvc5
结果: sat
解: x = 4, y = 1
验证: 4 + 1 = 5


---

## 3. SAT求解专题

**CNF公式（Conjunctive Normal Form）：**
```
(a ∨ b) ∧ (¬a ∨ c) ∧ (¬b ∨ ¬c ∨ d)
```

**三个场景：**
1. SAT - 可满足
2. UNSAT - 不可满足  
3. 逻辑谜题 - 说谎者问题

In [23]:
print("="*60)
print("场景1: SAT - 可满足")
print("="*60)

# 布尔变量
a, b, c = Bools('a b c')

solver = Solver()
solver.add(Or(a, b))
solver.add(Or(Not(a), c))
solver.add(Or(Not(b), Not(c)))

result = solver.check()
print(f"结果: {result}")

if result == sat:
    model = solver.model()
    print(f"解: a={model[a]}, b={model[b]}, c={model[c]}")

print("\n" + "="*60)
print("场景2: UNSAT - 不可满足")
print("="*60)

solver2 = Solver()
solver2.add(a)      # a = True
solver2.add(Not(a)) # a = False (矛盾!)

result2 = solver2.check()
print(f"结果: {result2}")
print("原因: 要求a同时为True和False")

print("\n" + "="*60)
print("场景3: 逻辑谜题 - 说谎者问题（稳定可展示版）")
print("="*60)
print("设 a/b/c 表示 A/B/C 是否说真话：True=真话, False=说谎")
print("A说: 'B 和 C 至少一个在说谎'")
print("B说: 'A 和 C 至少一个在说谎'")
print("C说: 'A 和 B 至少一个在说谎'")

solver3 = Solver()

# 三个人的陈述（都不涉及“我是不是说谎”的自指，避免悖论）
stmt_A = Or(Not(b), Not(c))  # B或C至少一个说谎
stmt_B = Or(Not(a), Not(c))  # A或C至少一个说谎
stmt_C = Or(Not(a), Not(b))  # A或B至少一个说谎

# “说真话 <-> 陈述为真；说谎 <-> 陈述为假”
solver3.add(a == stmt_A)
solver3.add(b == stmt_B)
solver3.add(c == stmt_C)

result3 = solver3.check()
print(f"结果: {result3}")

if result3 == sat:
    model = solver3.model()
    print(f"模型: a={model[a]}, b={model[b]}, c={model[c]}")
    print(f"解读: A={'真话' if is_true(model[a]) else '说谎'}, "
          f"B={'真话' if is_true(model[b]) else '说谎'}, "
          f"C={'真话' if is_true(model[c]) else '说谎'}")
else:
    print("无解：该设定下三个人的话无法同时自洽。")


场景1: SAT - 可满足
结果: sat
解: a=True, b=False, c=True

场景2: UNSAT - 不可满足
结果: unsat
原因: 要求a同时为True和False

场景3: 逻辑谜题 - 说谎者问题（稳定可展示版）
设 a/b/c 表示 A/B/C 是否说真话：True=真话, False=说谎
A说: 'B 和 C 至少一个在说谎'
B说: 'A 和 C 至少一个在说谎'
C说: 'A 和 B 至少一个在说谎'
结果: sat
模型: a=False, b=True, c=True
解读: A=说谎, B=真话, C=真话


---

## 4. 整数规划

**问题：** 最小化 f(x,y,z) = x² + y² + z²

**约束：**
```
x + y + z < 15
x + 3y + 2z > 10  
x, y, z >= 0
```

**方法：** 枚举目标值，找最小的可行解

In [4]:
print("="*60)
print("整数规划：最小化 x² + y² + z²")
print("="*60)

x, y, z = Ints('x y z')

base_constraints = [
    x + y + z < 15,
    x + 3*y + 2*z > 10,
    x >= 0, y >= 0, z >= 0
]

print("约束: x+y+z<15, x+3y+2z>10, x,y,z>=0\n")

# 枚举目标值
for target in range(50):
    solver = Solver()
    solver.add(base_constraints)
    solver.add(x*x + y*y + z*z == target)
    
    if solver.check() == sat:
        model = solver.model()
        xv, yv, zv = model[x].as_long(), model[y].as_long(), model[z].as_long()
        print(f"最小目标值: {target}")
        print(f"解: x={xv}, y={yv}, z={zv}")
        print(f"验证: {xv}²+{yv}²+{zv}² = {xv*xv + yv*yv + zv*zv}")
        break

整数规划：最小化 x² + y² + z²
约束: x+y+z<15, x+3y+2z>10, x,y,z>=0

最小目标值: 9
解: x=1, y=2, z=2
验证: 1²+2²+2² = 9


---

## 5. 量词逻辑

**全称量词 ∀** 和 **存在量词 ∃**

**问题：** 验证 ∀x. (x > 50 → ∃y. (y > 50 ∧ y > x))

含义：对所有x>50，都存在y使得y>50且y>x

In [24]:
print("="*60)
print("量词验证")
print("="*60)

x, y = Ints('x y')

# 构建公式
inner_exists = Exists([y], And(y > 50, y > x))
formula = ForAll([x], Implies(x > 50, inner_exists))

solver = Solver()
solver.add(formula)

result = solver.check()
print(f"验证 ∀x>50. ∃y. (y>50 ∧ y>x)")
print(f"结果: {result}")
print("解释: 整数无上界，总能找到更大的y")

量词验证
验证 ∀x>50. ∃y. (y>50 ∧ y>x)
结果: sat
解释: 整数无上界，总能找到更大的y


---

## 6. 数组理论

**操作：**
- Select(arr, i): 读 arr[i]
- Store(arr, i, v): 写 arr[i] = v

**问题：** 验证数组已排序性质

In [25]:
print("="*60)
print("数组理论：验证排序数组")
print("="*60)

arr = Array('arr', IntSort(), IntSort())

solver = Solver()

# 约束：3元素已排序
solver.add(Select(arr, 0) <= Select(arr, 1))
solver.add(Select(arr, 1) <= Select(arr, 2))
solver.add(Select(arr, 0) >= 1)
solver.add(Select(arr, 2) <= 10)
solver.add(Distinct(Select(arr, 0), Select(arr, 1), Select(arr, 2)))

result = solver.check()
if result == sat:
    model = solver.model()
    v0 = model.eval(Select(arr, 0)).as_long()
    v1 = model.eval(Select(arr, 1)).as_long()
    v2 = model.eval(Select(arr, 2)).as_long()
    print(f"找到排序数组: [{v0}, {v1}, {v2}]")

数组理论：验证排序数组
找到排序数组: [1, 2, 3]


---

## 7. 数独求解器

9x9网格，填入1-9，满足：
- 每行不重复
- 每列不重复
- 每个3x3宫不重复

In [26]:
def print_sudoku(grid, title):
    print(f"\n{title}")
    print("="*37)
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("  " + "-" * 33)
        row_str = "  "
        for j in range(9):
            if j % 3 == 0 and j != 0:
                row_str += "| "
            row_str += (str(grid[i][j]) if grid[i][j] != 0 else ".") + " "
        print(row_str)
    print()

# 初始谜题
puzzle = [
    [5,3,0, 0,7,0, 0,0,0],
    [6,0,0, 1,9,5, 0,0,0],
    [0,9,8, 0,0,0, 0,6,0],
    [8,0,0, 0,6,0, 0,0,3],
    [4,0,0, 8,0,3, 0,0,1],
    [7,0,0, 0,2,0, 0,0,6],
    [0,6,0, 0,0,0, 2,8,0],
    [0,0,0, 4,1,9, 0,0,5],
    [0,0,0, 0,8,0, 0,7,9]
]

print_sudoku(puzzle, "初始谜题")

# 创建变量
grid = [[Int(f'c_{i}_{j}') for j in range(9)] for i in range(9)]
solver = Solver()

# 域约束
for i in range(9):
    for j in range(9):
        solver.add(grid[i][j] >= 1, grid[i][j] <= 9)

# 已知格子
for i in range(9):
    for j in range(9):
        if puzzle[i][j] != 0:
            solver.add(grid[i][j] == puzzle[i][j])

# 行约束
for i in range(9):
    solver.add(Distinct(*grid[i]))

# 列约束
for j in range(9):
    solver.add(Distinct(*[grid[i][j] for i in range(9)]))

# 宫约束
for br in range(3):
    for bc in range(3):
        box = [grid[3*br+i][3*bc+j] for i in range(3) for j in range(3)]
        solver.add(Distinct(*box))

# 求解
start = time.time()
result = solver.check()
elapsed = time.time() - start

if result == sat:
    model = solver.model()
    solution = [[model[grid[i][j]].as_long() for j in range(9)] for i in range(9)]
    print_sudoku(solution, f"解 (耗时{elapsed:.3f}秒)")


初始谜题
  5 3 . | . 7 . | . . . 
  6 . . | 1 9 5 | . . . 
  . 9 8 | . . . | . 6 . 
  ---------------------------------
  8 . . | . 6 . | . . 3 
  4 . . | 8 . 3 | . . 1 
  7 . . | . 2 . | . . 6 
  ---------------------------------
  . 6 . | . . . | 2 8 . 
  . . . | 4 1 9 | . . 5 
  . . . | . 8 . | . 7 9 


解 (耗时5.385秒)
  5 3 4 | 6 7 8 | 9 1 2 
  6 7 2 | 1 9 5 | 3 4 8 
  1 9 8 | 3 4 2 | 5 6 7 
  ---------------------------------
  8 5 9 | 7 6 1 | 4 2 3 
  4 2 6 | 8 5 3 | 7 9 1 
  7 1 3 | 9 2 4 | 8 5 6 
  ---------------------------------
  9 6 1 | 5 3 7 | 2 8 4 
  2 8 7 | 4 1 9 | 6 3 5 
  3 4 5 | 2 8 6 | 1 7 9 



---

## 8. N皇后问题

N个皇后放在NxN棋盘，任意两个不互相攻击。

使用N个整数变量q[i]表示第i行皇后的列位置。

In [27]:
print("="*60)
print("N皇后问题 (N=8)")
print("="*60)

N = 8
queens = [Int(f'q_{i}') for i in range(N)]

solver = Solver()

# 域约束
for i in range(N):
    solver.add(queens[i] >= 0, queens[i] < N)

# 列约束
solver.add(Distinct(*queens))

# 对角线约束
for i in range(N):
    for j in range(i + 1, N):
        solver.add(queens[i] - queens[j] != i - j)
        solver.add(queens[i] - queens[j] != j - i)

start = time.time()
result = solver.check()
elapsed = time.time() - start

if result == sat:
    model = solver.model()
    solution = [model[queens[i]].as_long() for i in range(N)]
    print(f"找到解: {solution}")
    print(f"耗时: {elapsed:.3f}秒")
    
    # 打印棋盘
    print("\n棋盘:")
    for i in range(N):
        row = ""
        for j in range(N):
            row += "Q " if solution[i] == j else ". "
        print("  " + row)

N皇后问题 (N=8)
找到解: [0, 4, 7, 5, 2, 6, 1, 3]
耗时: 0.100秒

棋盘:
  Q . . . . . . . 
  . . . . Q . . . 
  . . . . . . . Q 
  . . . . . Q . . 
  . . Q . . . . . 
  . . . . . . Q . 
  . Q . . . . . . 
  . . . Q . . . . 


---

## 9. 有界模型检查(BMC)

**Bounded Model Checking**: 验证系统在k步内不违反性质。

**交通灯系统:**
```
状态: Red(0) → Green(1) → Yellow(2) → Red
性质: 永不从Red直接跳到Yellow
```

**BMC公式:**
```
I(s₀) ∧ T(s₀,s₁) ∧ ... ∧ T(sₖ₋₁,sₖ) ∧ ¬P(s₀,...,sₖ)
```

In [28]:
print("="*60)
print("BMC: 交通灯系统验证")
print("="*60)

RED, GREEN, YELLOW = 0, 1, 2
K = 10

states = [Int(f's_{i}') for i in range(K + 1)]
solver = Solver()

# 初始状态: Red
solver.add(states[0] == RED)

# 转换关系
for i in range(K):
    solver.add(Or(
        And(states[i] == RED, states[i+1] == GREEN),
        And(states[i] == GREEN, states[i+1] == YELLOW),
        And(states[i] == YELLOW, states[i+1] == RED)
    ))

# 性质否定: 寻找Red→Yellow的反例
violations = [And(states[i] == RED, states[i+1] == YELLOW) for i in range(K)]
solver.add(Or(*violations))

result = solver.check()
print(f"验证{K}步内是否违反性质")
print(f"结果: {result}")

if result == unsat:
    print(f"性质成立: {K}步内不会从Red跳到Yellow")
else:
    print("发现违反路径")

BMC: 交通灯系统验证
验证10步内是否违反性质
结果: unsat
性质成立: 10步内不会从Red跳到Yellow


---

## 9.1 实战：硬件计数器验证

使用 BMC 验证一个 4 位计数器的正确性。

**计数器行为：**
- 每个时钟周期递增 1
- 复位时清零
- 4 位，范围 0-15

**验证属性：**
在复位释放后的第 2 个时钟周期，输出应该等于 2

In [29]:
print("="*70)
print("硬件计数器验证：使用 BMC 验证 4 位计数器")
print("="*70)
print()

print("【1. 硬件设计】")
print("-" * 70)
print("""
Verilog 计数器设计：
  module counter(
      input clk,
      input rst_n,        // 低电平复位
      output reg [3:0] out
  );
      always @(posedge clk) begin
          if (rst_n == 0)
              out <= 4'b0000;  // 复位：清零
          else
              out <= out + 1;  // 正常：递增
      end
  endmodule

功能：
  - 每个时钟上升沿，计数器递增 1
  - rst_n=0 时，强制清零
  - 4 位输出，范围 0-15（溢出后回到 0）
""")

print()
print("【2. 建模：将硬件转换为 SMT 公式】")
print("-" * 70)

# 参数
K = 10  # 验证 10 个时钟周期
WIDTH = 4  # 4 位计数器

# 为每个时刻创建符号变量
# out[0], out[1], ..., out[K]: 每个时刻的计数器值
out = [BitVec(f'out_{i}', WIDTH) for i in range(K + 1)]

# 输入信号：每个时刻的复位信号
rst_n = [Bool(f'rst_n_{i}') for i in range(K)]

print(f"创建变量：")
print(f"  - 状态变量 out[0..{K}]: 每个时刻的计数器值（4位）")
print(f"  - 输入变量 rst_n[0..{K-1}]: 每个时刻的复位信号")
print()

print("【3. 添加约束：初始状态 + 转换关系】")
print("-" * 70)

solver = Solver()

# 约束 1: 初始状态
# 计数器初始值为 0
initial_state = (out[0] == 0)
solver.add(initial_state)
print("✓ 初始状态: out[0] = 0")

# 约束 2: 转换关系（模拟 always 块）
# 对每个时刻 i，定义 out[i+1] 如何从 out[i] 转换
print("✓ 转换关系: 模拟 Verilog always @(posedge clk)")
for i in range(K):
    # Verilog: if (rst_n == 0) out <= 0; else out <= out + 1;
    # SMT: out[i+1] = If(rst_n[i], out[i] + 1, 0)
    transition = (
        out[i+1] == If(rst_n[i],           # 如果 rst_n=1 (未复位)
                       out[i] + 1,         # 则递增
                       BitVecVal(0, WIDTH)) # 否则清零
    )
    solver.add(transition)
    
print(f"  添加了 {K} 个转换约束")
print()

print("【4. 设置测试场景】")
print("-" * 70)
print("场景：复位 5 个周期，然后释放")
print()

# 步骤 0-4: 复位（rst_n = 0）
for i in range(5):
    solver.add(rst_n[i] == False)
    print(f"  步骤 {i}: rst_n = 0 (复位中)")

print()

# 步骤 5-9: 释放复位（rst_n = 1）
for i in range(5, K):
    solver.add(rst_n[i] == True)
    if i == 5:
        print(f"  步骤 {i}: rst_n = 1 (释放复位，开始计数)")
    else:
        print(f"  步骤 {i}: rst_n = 1")

print()
print("【5. 验证属性】")
print("-" * 70)
print("属性：在步骤 7，计数器输出应该等于 2")
print()
print("分析：")
print("  步骤 0-4: 复位，out 保持 0")
print("  步骤 5: 释放复位，out 仍为 0")
print("  步骤 6: 第 1 个上升沿，out = 0 + 1 = 1")
print("  步骤 7: 第 2 个上升沿，out = 1 + 1 = 2 ← 验证这个")
print()

# 添加属性
property_step = 7
solver.add(out[property_step] != 2)  # 寻找反例

# 执行验证
start = time.time()
result = solver.check()
elapsed = time.time() - start

print(f"BMC 验证结果: {result}")
print(f"求解耗时: {elapsed:.6f} 秒")
print()

print("【6. 分析结果】")
print("-" * 70)

if result == sat:
    model = solver.model()
    
    print("✓ 验证通过！找到满足属性的执行轨迹：")
    print()
    print("时刻 | rst_n | out (二进制) | out (十进制) | 说明")
    print("-----|-------|--------------|--------------|" + "-"*20)
    
    for i in range(K + 1):
        # 获取变量值
        out_val = model[out[i]].as_long()
        out_bin = format(out_val, '04b')
        
        if i < K:
            rst_val = "0" if not is_true(model[rst_n[i]]) else "1"
        else:
            rst_val = "-"
        
        # 添加说明
        if i <= 4:
            note = "复位中，保持 0"
        elif i == 5:
            note = "复位释放"
        elif i == 6:
            note = "第 1 次递增"
        elif i == 7:
            note = "第 2 次递增 ✓"
        else:
            note = f"第 {i-5} 次递增"
        
        print(f"  {i:2d} |   {rst_val}   |     {out_bin}     |      {out_val:2d}      | {note}")
    
    print()
    print("="*70)
    print("结论")
    print("="*70)
    print(f"✓ 在步骤 {property_step}，计数器输出确实为 2")
    print("✓ 硬件设计符合预期行为")
    print("✓ BMC 验证成功")
else:
    print("✗ 验证失败：无法找到满足属性的执行路径")
    print("  可能原因：")
    print("  - 硬件设计有 bug")
    print("  - 测试场景设置错误")
    print("  - 属性本身不可达")

print()
print("="*70)
print("BMC 验证原理总结")
print("="*70)
print("""
有界模型检查 (Bounded Model Checking) 的核心思想：

1. 展开 (Unrolling)：
   将时序电路展开 k 步
   I(s₀) ∧ T(s₀,s₁) ∧ T(s₁,s₂) ∧ ... ∧ T(sₖ₋₁,sₖ)

2. 属性检查：
   询问 SMT 求解器：是否存在一条路径使得属性成立？
   
3. 结果解释：
   - SAT: 找到一条满足属性的路径（本例中找到了）
   - UNSAT: 在 k 步内没有违反属性

完整的 Verilog-BMC 工具链（参考项目）：
  Verilog源码 → Yosys综合 → BTOR2中间格式 → PySMT建模 → cvc5验证
""")

硬件计数器验证：使用 BMC 验证 4 位计数器

【1. 硬件设计】
----------------------------------------------------------------------

Verilog 计数器设计：
  module counter(
      input clk,
      input rst_n,        // 低电平复位
      output reg [3:0] out
  );
      always @(posedge clk) begin
          if (rst_n == 0)
              out <= 4'b0000;  // 复位：清零
          else
              out <= out + 1;  // 正常：递增
      end
  endmodule

功能：
  - 每个时钟上升沿，计数器递增 1
  - rst_n=0 时，强制清零
  - 4 位输出，范围 0-15（溢出后回到 0）


【2. 建模：将硬件转换为 SMT 公式】
----------------------------------------------------------------------
创建变量：
  - 状态变量 out[0..10]: 每个时刻的计数器值（4位）
  - 输入变量 rst_n[0..9]: 每个时刻的复位信号

【3. 添加约束：初始状态 + 转换关系】
----------------------------------------------------------------------
✓ 初始状态: out[0] = 0
✓ 转换关系: 模拟 Verilog always @(posedge clk)
  添加了 10 个转换约束

【4. 设置测试场景】
----------------------------------------------------------------------
场景：复位 5 个周期，然后释放

  步骤 0: rst_n = 0 (复位中)
  步骤 1: rst_n = 0 (复位中)
  步骤 2: rst_n = 0 (复位中)
  步骤 3: rst_n = 

---

## 10. DPLL(T)算法内部机制

DPLL(T) = DPLL(布尔层) + T(理论层)

**执行流程:**
```
[1] 布尔抽象
    理论约束 → 布尔变量
    
[2] DPLL求解
    找一个布尔赋值
    
[3] 理论检查
    该赋值在理论层是否一致?
    
[4] 冲突学习
    if 不一致: 学习冲突子句，回溯
    if 一致: 输出模型
```

**示例:** 验证 if(x>0) then y=5 else y=10

In [34]:
print("="*70)
print("DPLL(T)执行过程演示（带分支选择）")
print("="*70)

x, y, z = Ints('x y z')
b1, b2 = Bools('b1 b2')

solver = Solver()

# 约束: if(x>0) then y=5 else y=10
solver.add(b1 == (x > 0))
solver.add(Implies(b1, y == 5))
solver.add(Implies(Not(b1), y == 10))

# 约束: if(y==5) then z>100 else z<50
solver.add(b2 == (y == 5))
solver.add(Implies(b2, z > 100))
solver.add(Implies(Not(b2), z < 50))

print("[阶段1] 布尔抽象：b1 表示 (x>0)，b2 表示 (y==5)")
print()

print("[阶段2] DPLL选择分支：先假设 b1=True, b2=True")
solver.push()
solver.add(b1)  # 强行选择 b1=True
solver.add(b2)  # 强行选择 b2=True

print("[阶段3] 理论检查：在该分支下求解")
result = solver.check()
print("  结果:", result)

if result == sat:
    m = solver.model()
    print("  模型: x =", m[x].as_long(), ", y =", m[y].as_long(), ", z =", m[z].as_long())
solver.pop()

print("\n再试另一个分支：b1=False, b2=False")
solver.push()
solver.add(Not(b1))
solver.add(Not(b2))
result = solver.check()
print("  结果:", result)
if result == sat:
    m = solver.model()
    print("  模型: x =", m[x].as_long(), ", y =", m[y].as_long(), ", z =", m[z].as_long())
solver.pop()


DPLL(T)执行过程演示（带分支选择）
[阶段1] 布尔抽象：b1 表示 (x>0)，b2 表示 (y==5)

[阶段2] DPLL选择分支：先假设 b1=True, b2=True
[阶段3] 理论检查：在该分支下求解
  结果: sat
  模型: x = 1 , y = 5 , z = 101

再试另一个分支：b1=False, b2=False
  结果: sat
  模型: x = 0 , y = 10 , z = 49


---

## 11. UNSAT Core冲突分析

当约束不可满足时，UNSAT Core是导致矛盾的最小约束子集。

**场景:** 排班系统
- 3人分配到不同班次
- 只有2个可选班次
- 鸽巢原理: 不可能!

In [35]:
print("="*70)
print("UNSAT Core: 排班冲突诊断")
print("="*70)

alice, bob, charlie = Ints('alice bob charlie')

print("约束逐步添加:")
print()

# 阶段1: 基础
solver = Solver()
solver.add(alice >= 1, alice <= 4)
solver.add(bob >= 1, bob <= 4)
solver.add(charlie >= 1, charlie <= 4)
print(f"[1] 域约束(1-4班次): {solver.check()}")

# 阶段2: 互斥
solver.add(alice != bob)
solver.add(bob != charlie)
solver.add(alice != charlie)
print(f"[2] + 互斥约束: {solver.check()}")

# 阶段3: Alice要求
solver.add(alice <= 2)
print(f"[3] + Alice只要早/中班: {solver.check()}")

# 阶段4: Bob要求
solver.add(bob <= 2)
print(f"[4] + Bob只要早/中班: {solver.check()}")

# 阶段5: Charlie要求 → UNSAT
solver.add(charlie <= 2)
result = solver.check()
print(f"[5] + Charlie也只要早/中班: {result}")
print()

if result == unsat:
    print("冲突分析:")
    print("  导致矛盾的约束:")
    print("  - alice ≠ bob ≠ charlie (互斥)")
    print("  - alice ≤ 2, bob ≤ 2, charlie ≤ 2")
    print("  ")
    print("  原因: 3个人争抢2个班次")


UNSAT Core: 排班冲突诊断
约束逐步添加:

[1] 域约束(1-4班次): sat
[2] + 互斥约束: sat
[3] + Alice只要早/中班: sat
[4] + Bob只要早/中班: sat
[5] + Charlie也只要早/中班: unsat

冲突分析:
  导致矛盾的约束:
  - alice ≠ bob ≠ charlie (互斥)
  - alice ≤ 2, bob ≤ 2, charlie ≤ 2
  
  原因: 3个人争抢2个班次


---

## 12. 增量求解与状态管理

push/pop机制复用之前的推理，避免重复计算。

**对比:**
```
非增量: 每次新建Solver → 重复初始化
增量:   push添加约束 → check → pop回退
```

In [37]:
print("="*70)
print("增量求解: 云服务配置探索")
print("="*70)

cpu, mem = Ints('cpu mem')

base = [cpu >= 1, cpu <= 64, mem >= cpu * 2, mem <= 512]

scenarios = [
    ("Web服务器", [cpu <= 8, mem <= 32]),
    ("数据库", [cpu >= 16, mem >= 128]),
    ("AI训练", [cpu >= 32, mem >= 256]),
]

print("方法1: 非增量")
times_non_inc = []
for name, constraints in scenarios:
    solver = Solver()  # 每次新建
    solver.add(base)
    solver.add(constraints)
    start = time.time()
    result = solver.check()
    elapsed = time.time() - start
    times_non_inc.append(elapsed)
    print(f"  {name:12s}: {elapsed:.6f}s, {result}")

print("\n方法2: 增量(push/pop)")
solver_shared = Solver()
solver_shared.add(base)
times_inc = []
for name, constraints in scenarios:
    solver_shared.push()  # 保存状态
    solver_shared.add(constraints)
    start = time.time()
    result = solver_shared.check()
    elapsed = time.time() - start
    times_inc.append(elapsed)
    print(f"  {name:12s}: {elapsed:.6f}s, {result}")
    solver_shared.pop()   # 恢复状态

total_non_inc = sum(times_non_inc)
total_inc = sum(times_inc)
speedup = total_non_inc / total_inc if total_inc > 0 else 1

print(f"\n非增量总耗时: {total_non_inc:.6f}s")
print(f"增量总耗时:   {total_inc:.6f}s")
print(f"加速比:       {speedup:.2f}x")

增量求解: 云服务配置探索
方法1: 非增量
  Web服务器      : 0.000917s, sat
  数据库         : 0.000953s, sat
  AI训练        : 0.000651s, sat

方法2: 增量(push/pop)
  Web服务器      : 0.000441s, sat
  数据库         : 0.000245s, sat
  AI训练        : 0.000847s, sat

非增量总耗时: 0.002521s
增量总耗时:   0.001533s
加速比:       1.64x


---

## 13. 位向量硬件验证

验证4位加法器逐位正确性。

**全加器逻辑:**
```
sum[i]   = a[i] ⊕ b[i] ⊕ carry[i]
carry[i+1] = (a[i] ∧ b[i]) ∨ ((a[i] ⊕ b[i]) ∧ carry[i])
```

In [38]:
print("="*70)
print("位向量: 验证4位加法器")
print("="*70)

a = BitVec('a', 4)
b = BitVec('b', 4)

solver = Solver()

# 手工实现的加法器
c0 = BitVecVal(0, 1)
c1 = BitVec('c1', 1)
c2 = BitVec('c2', 1)
c3 = BitVec('c3', 1)

# 提取每一位
from cvc5.pythonic import Extract, Concat

a0, a1, a2, a3 = Extract(0,0,a), Extract(1,1,a), Extract(2,2,a), Extract(3,3,a)
b0, b1, b2, b3 = Extract(0,0,b), Extract(1,1,b), Extract(2,2,b), Extract(3,3,b)

# 全加器
s0 = a0 ^ b0 ^ c0
solver.add(c1 == ((a0 & b0) | ((a0 ^ b0) & c0)))

s1 = a1 ^ b1 ^ c1
solver.add(c2 == ((a1 & b1) | ((a1 ^ b1) & c1)))

s2 = a2 ^ b2 ^ c2
solver.add(c3 == ((a2 & b2) | ((a2 ^ b2) & c2)))

s3 = a3 ^ b3 ^ c3

manual_sum = Concat(s3, Concat(s2, Concat(s1, s0)))

# 验证: 手工实现 ≠ 内置加法器 (寻找反例)
solver.add(manual_sum != Extract(3, 0, a + b))

start = time.time()
result = solver.check()
elapsed = time.time() - start

print(f"验证策略: 寻找反例 (手工 ≠ 内置)")
print(f"结果: {result}")

if result == unsat:
    print("验证通过! 在所有2^8=256种输入下等价")
    print(f"耗时: {elapsed:.6f}秒")
else:
    print("发现反例")

位向量: 验证4位加法器
验证策略: 寻找反例 (手工 ≠ 内置)
结果: unsat
验证通过! 在所有2^8=256种输入下等价
耗时: 0.001770秒


---

## 14. 量词实例化优化

**E-matching触发器机制**

朴素量词可能导致指数级实例化，显式实例化更高效。

**对比:** 验证数组单调性 ∀i,j. i<j → arr[i]≤arr[j]

In [39]:
print("="*70)
print("量词优化: 数组单调性")
print("="*70)

arr = Array('arr', IntSort(), IntSort())
n = 5

# 具体数组
arr_vals = [1, 3, 5, 7, 9]

print("方法1: 朴素量词")
solver1 = Solver()
for i in range(n):
    solver1.add(Select(arr, i) == arr_vals[i])

i_q, j_q = Ints('i_q j_q')
solver1.add(ForAll([i_q, j_q], 
    Implies(And(0 <= i_q, i_q < j_q, j_q < n),
            Select(arr, i_q) <= Select(arr, j_q))))

start = time.time()
result1 = solver1.check()
elapsed1 = time.time() - start
print(f"  结果: {result1}, 耗时: {elapsed1:.6f}s")

print("\n方法2: 显式实例化")
solver2 = Solver()
for i in range(n):
    solver2.add(Select(arr, i) == arr_vals[i])

# 显式展开所有(i,j)对
for i in range(n):
    for j in range(i+1, n):
        solver2.add(Select(arr, i) <= Select(arr, j))

start = time.time()
result2 = solver2.check()
elapsed2 = time.time() - start
print(f"  结果: {result2}, 耗时: {elapsed2:.6f}s")

speedup = elapsed1 / elapsed2 if elapsed2 > 0 else 1
print(f"\n加速比: {speedup:.2f}x")
print(f"原因: 避免触发器匹配开销")

量词优化: 数组单调性
方法1: 朴素量词
  结果: unknown, 耗时: 0.009907s

方法2: 显式实例化
  结果: sat, 耗时: 0.020440s

加速比: 0.48x
原因: 避免触发器匹配开销


---

## 15. 程序正确性验证

Hoare逻辑: {前置条件} 代码 {后置条件}

**验证二分查找:** 如果返回索引i，则arr[i]=target

In [40]:
print("="*70)
print("程序验证: 二分查找")
print("="*70)

arr = Array('arr', IntSort(), IntSort())
target, left, right, mid, result = Ints('target left right mid result')

solver = Solver()

# 前置条件
n = 7
for i in range(n):
    solver.add(Select(arr, i) == 2*i + 1)  # [1,3,5,7,9,11,13]

solver.add(left >= 0, left <= right, right < n)
solver.add(target >= Select(arr, left))
solver.add(target <= Select(arr, right))

# 程序逻辑
solver.add(mid == (left + right) / 2)
solver.add(Select(arr, mid) == target)
solver.add(result == mid)

# 验证: 寻找反例 arr[result] ≠ target
solver.add(Select(arr, result) != target)

result_check = solver.check()
print(f"验证: 如果返回result，则arr[result]=target")
print(f"寻找反例: arr[result]≠target")
print(f"结果: {result_check}")

if result_check == unsat:
    print("验证通过! 后置条件成立")

程序验证: 二分查找
验证: 如果返回result，则arr[result]=target
寻找反例: arr[result]≠target
结果: unsat
验证通过! 后置条件成立


---

## 16. 递归数据结构验证

验证链表反转的对合性质: reverse(reverse(L)) = L

**数学归纳:**
```
基础: reverse(reverse([])) = []
归纳: 假设性质对L成立，证明对x::L也成立
```

In [41]:
print("="*70)
print("数据结构: 链表反转验证")
print("="*70)

# 用数组模拟链表
arr_orig = Array('arr_orig', IntSort(), IntSort())
arr_rev1 = Array('arr_rev1', IntSort(), IntSort())
arr_rev2 = Array('arr_rev2', IntSort(), IntSort())

n = 5
solver = Solver()

# 原始链表
for i in range(n):
    solver.add(Select(arr_orig, i) == i + 1)  # [1,2,3,4,5]

# 反转一次
for i in range(n):
    solver.add(Select(arr_rev1, i) == Select(arr_orig, n-1-i))

# 反转两次
for i in range(n):
    solver.add(Select(arr_rev2, i) == Select(arr_rev1, n-1-i))

# 验证: reverse(reverse(L)) ≠ L (寻找反例)
violations = [Select(arr_rev2, i) != Select(arr_orig, i) for i in range(n)]
solver.add(Or(*violations))

result = solver.check()
print(f"验证: reverse(reverse(L)) = L")
print(f"寻找反例: reverse(reverse(L)) ≠ L")
print(f"结果: {result}")

if result == unsat:
    print("验证通过! reverse是对合(involution)")
    print("数学意义: reverse ∘ reverse = identity")

数据结构: 链表反转验证
验证: reverse(reverse(L)) = L
寻找反例: reverse(reverse(L)) ≠ L
结果: unsat
验证通过! reverse是对合(involution)
数学意义: reverse ∘ reverse = identity


---

## 17. LLM + cvc5: 自然语言约束求解器

**创新点:** 将大语言模型与SMT求解器结合,降低使用门槛

**技术架构:**
```
用户输入(中文自然语言)
    ↓
LLM (GPT-4 / gpt-5.1)
    ↓
cvc5 Python代码生成
    ↓
自动执行求解
    ↓
返回结果
```

**核心价值:**
- LLM: 理解自然语言,生成代码
- SMT: 保证逻辑严谨性,精确求解
- 协同: "易用性" + "可靠性"

In [42]:
print("="*70)
print("LLM + cvc5 集成演示")
print("="*70)
print()

# 导入LLM求解器模块

import sys
sys.path.append('../examples/llm_integration')

from nl2smt_solver import call_llm, extract_code, execute_code

print("功能: 用中文描述问题,自动生成cvc5代码并求解")
print()
print("示例问题:")
print("1. 找三个正整数,和为15,第一个数是偶数")
print("2. 张三比李四大5岁,两人年龄和是35,问张三多少岁")  
print("3. 找一个两位数,个位是十位的2倍,能被3整除")
print()
print("-"*70)

LLM + cvc5 集成演示

功能: 用中文描述问题,自动生成cvc5代码并求解

示例问题:
1. 找三个正整数,和为15,第一个数是偶数
2. 张三比李四大5岁,两人年龄和是35,问张三多少岁
3. 找一个两位数,个位是十位的2倍,能被3整除

----------------------------------------------------------------------


### 示例1: 整数约束问题

**问题:** 找三个正整数,和为15,第一个数是偶数

In [43]:
query1 = "找三个正整数,和为15,第一个数是偶数"

print(f"问题: {query1}")
print()
print("【步骤1】调用LLM生成代码...")

# 调用LLM生成代码
llm_response = call_llm(query1)
code = extract_code(llm_response)

print("【步骤2】生成的cvc5代码:")
print("-"*70)
print(code)
print("-"*70)

print()
print("【步骤3】执行代码,求解:")
print("-"*70)
execute_code(code)
print("-"*70)

问题: 找三个正整数,和为15,第一个数是偶数

【步骤1】调用LLM生成代码...
【步骤2】生成的cvc5代码:
----------------------------------------------------------------------
from cvc5.pythonic import *

# 定义三个正整数
x, y, z = Ints('x y z')

s = Solver()

# 约束条件：
# 1) 都是正整数
# 2) 和为 15
# 3) 第一个数是偶数
s.add(x > 0, y > 0, z > 0)
s.add(x + y + z == 15)
s.add(x % 2 == 0)

if s.check() == sat:
    m = s.model()
    print("x =", m[x], "y =", m[y], "z =", m[z])
else:
    print("无解")
----------------------------------------------------------------------

【步骤3】执行代码,求解:
----------------------------------------------------------------------
x = 2 y = 12 z = 1
----------------------------------------------------------------------


### 示例2: 逻辑推理问题

**问题:** 张三比李四大5岁,两人年龄和是35,问张三多少岁

In [44]:
query2 = "张三比李四大5岁,两人年龄和是35,问张三多少岁"

print(f"问题: {query2}")
print()

# 调用LLM并执行
llm_response = call_llm(query2)
code = extract_code(llm_response)

print("生成的代码:")
print("-"*70)
print(code)
print("-"*70)

print("\n求解结果:")
print("-"*70)
execute_code(code)
print("-"*70)

问题: 张三比李四大5岁,两人年龄和是35,问张三多少岁

生成的代码:
----------------------------------------------------------------------
from cvc5.pythonic import *

# 定义整数变量：张三和李四的年龄
zhangsan, lisi = Ints('zhangsan lisi')

s = Solver()

# 约束：张三比李四大 5 岁
s.add(zhangsan == lisi + 5)

# 约束：两人年龄和是 35
s.add(zhangsan + lisi == 35)

if s.check() == sat:
    m = s.model()
    print("张三的年龄:", m[zhangsan])
else:
    print("无解")
----------------------------------------------------------------------

求解结果:
----------------------------------------------------------------------
张三的年龄: 20
----------------------------------------------------------------------


### 示例3: 数学问题

**问题:** 找一个两位数,个位是十位的2倍,能被3整除

In [45]:
query3 = "找一个两位数,个位是十位的2倍,能被3整除"

print(f"问题: {query3}")
print()

llm_response = call_llm(query3)
code = extract_code(llm_response)

print("生成的代码:")
print("-"*70)
print(code)
print("-"*70)

print("\n所有满足条件的两位数:")
print("-"*70)
execute_code(code)
print("-"*70)

问题: 找一个两位数,个位是十位的2倍,能被3整除

生成的代码:
----------------------------------------------------------------------
from cvc5.pythonic import *

# 定义变量：十位 t，个位 u，两位数 n
t, u = Ints('t u')
n = Int('n')

s = Solver()

# 两位数范围约束：十位 1..9，个位 0..9
s.add(t >= 1, t <= 9)
s.add(u >= 0, u <= 9)

# 个位是十位的 2 倍
s.add(u == 2 * t)

# n 为对应的两位数
s.add(n == 10 * t + u)

# n 能被 3 整除
s.add(n % 3 == 0)

if s.check() == sat:
    m = s.model()
    print(m.eval(n))
else:
    print("无解")
----------------------------------------------------------------------

所有满足条件的两位数:
----------------------------------------------------------------------
12
----------------------------------------------------------------------


### 技术实现细节

**1. Few-Shot Prompting**
- 在提示词中提供3个示例
- 教会LLM生成正确的cvc5代码格式
- 包含变量定义、约束添加、求解流程

**2. 代码提取与执行**
- 从LLM响应中提取```python代码块
- 处理if __name__ == "__main__"结构
- 在独立命名空间中安全执行

**3. 错误处理**
- LLM生成失败: 提示重试
- 代码执行出错: 显示错误信息
- 超时保护: 避免无限循环

**4. 支持的后端**
- OpenAI兼容API (当前使用gpt-5.1)
- 本地Ollama (免费,可离线)

**完整代码:** `examples/llm_integration/`